In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import eigvals

# 1. 读取数据
df = pd.read_csv('Credit.csv')

# 2. 提取因变量和自变量（确保列名正确）
# 根据你提供的文件内容，列名为：
# 'Balance', 'Age', 'Cards', 'Education', 'Income', 'Limit', 'Rating'
y = df['Balance']
X_raw = df[['Age', 'Cards', 'Education', 'Income', 'Limit', 'Rating']]

# 3. 构造设计矩阵 X（不含截距项，因为条件指数通常针对原始变量）
# 如果你想包含截距项，可取消下一行注释：
# X = np.column_stack([np.ones(X_raw.shape[0]), X_raw.values])
X = X_raw.values  # 不含截距项

# 4. 计算 X'X
XtX = X.T @ X

# 5. 计算特征值（实部，排序）
eigenvalues = eigvals(XtX).real
# 排除非常小的负数（数值误差）
eigenvalues = eigenvalues[eigenvalues > 1e-10]
eigenvalues = np.sort(eigenvalues)[::-1]  # 降序

if len(eigenvalues) == 0:
    raise ValueError("所有特征值接近零，矩阵奇异。")

# 6. 计算条件指数
lambda_max = eigenvalues[0]
condition_indices = np.sqrt(lambda_max / eigenvalues)

# 7. 输出结果
print("=== 特征值 ===")
for i, ev in enumerate(eigenvalues):
    print(f"λ_{i}: {ev:.6e}")

print("\n=== 条件指数 (Condition Index) ===")
for i, ci in enumerate(condition_indices):
    if ci < 10:
        level = "无多重共线性"
    elif ci < 30:
        level = "中等或较强共线性"
    else:
        level = "严重多重共线性"
    print(f"k_{i}: {ci:8.2f} → {level}")

max_ci = condition_indices.max()
print(f"\n最大条件指数: {max_ci:.2f}")
if max_ci > 30:
    print("⚠️ 结论：存在严重的多重共线性！")
elif max_ci > 10:
    print("⚠️ 结论：存在中等或较强的多重共线性。")
else:
    print("✅ 结论：无显著多重共线性。")